# Data Cleaning dan Kombinasi CoreTax
## Cleaning data scraping baru dan menggabungkan dengan data yang sudah ada

**Data Input:**
1. `CoreTax Scraper M Pajak 2025.csv` - Play Store reviews
2. `Scraper Youtube CoreTax.csv` - YouTube comments
3. `Hackathon Sentiment Analysis Combined.csv` - Data yang sudah ada sebelumnya

**Output:**
- Data bersih yang sudah digabungkan dengan format yang konsisten

## 1. Setup dan Import Libraries

In [ ]:
import pandas as pd
import re
import warnings
warnings.filterwarnings('ignore')

print("✓ Libraries imported successfully!")

## 2. Mount Google Drive dan Setup Path

In [ ]:
from google.colab import drive
import os

print("Mounting Google Drive...")
drive.mount('/content/drive/')
print("✓ Google Drive mounted!")

# Set data path
DATA_PATH = '/content/drive/MyDrive/Hackathon/data/'

print(f"Data path: {DATA_PATH}")

# Cek apakah folder ada
if os.path.exists(DATA_PATH):
    print("✓ Data folder found!")
    print(f"\nFiles in data folder:")
    for file in os.listdir(DATA_PATH):
        print(f"  - {file}")
else:
    print("⚠️ Data folder not found! Please check the path.")

## 3. Fungsi Preprocessing

In [ ]:
def clean_text(text):
    """
    Fungsi untuk membersihkan teks
    """
    if pd.isna(text):
        return ""
    
    text = str(text)
    
    # Lowercase
    text = text.lower()
    
    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    
    # Remove mentions (@username)
    text = re.sub(r'@\w+', '', text)
    
    # Remove hashtags tapi simpan kata-katanya
    text = re.sub(r'#(\w+)', r'\1', text)
    
    # Remove email
    text = re.sub(r'\S+@\S+', '', text)
    
    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

print("✓ Fungsi preprocessing ready!")

## 4. Load Data yang Sudah Ada (Hackathon Sentiment Analysis Combined)

In [ ]:
print("=" * 80)
print("LOADING DATA YANG SUDAH ADA")
print("=" * 80)

try:
    FILE_NAME = 'Hackathon Sentiment Analysis Combined.csv'
    df_existing = pd.read_csv(DATA_PATH + FILE_NAME, encoding='utf-8')
    
    print(f"✓ Data existing loaded: {len(df_existing)} baris")
    print(f"Kolom: {df_existing.columns.tolist()}")
    print(f"\nPreview data:")
    display(df_existing.head())
    
    print(f"\nDistribusi source:")
    print(df_existing['source'].value_counts())
    
    if 'sentiment' in df_existing.columns:
        print(f"\nDistribusi sentiment:")
        print(df_existing['sentiment'].value_counts())
    
except Exception as e:
    print(f"⚠️ Error loading existing data: {e}")
    df_existing = pd.DataFrame()

## 5. Load dan Clean Data Play Store (M Pajak)

In [ ]:
print("=" * 80)
print("CLEANING DATA: CoreTax Scraper M Pajak 2025.csv")
print("=" * 80)

try:
    # Load data Play Store
    df_playstore = pd.read_csv(DATA_PATH + 'CoreTax Scraper M Pajak 2025.csv', encoding='utf-8')
    
    print(f"✓ Data Play Store loaded: {len(df_playstore)} baris")
    print(f"Kolom: {df_playstore.columns.tolist()}")
    print(f"\nPreview data:")
    display(df_playstore.head())
    
    # Rename kolom agar sesuai format
    # rating, at, content, source -> date, text, source
    df_playstore_clean = pd.DataFrame({
        'text': df_playstore['content'],
        'source': 'play_store'
    })
    
    # Tambahkan kolom date jika ada
    if 'at' in df_playstore.columns:
        df_playstore_clean['date'] = df_playstore['at']
    
    # Tambahkan kolom rating untuk referensi
    if 'rating' in df_playstore.columns:
        df_playstore_clean['rating'] = df_playstore['rating']
    
    # Clean text
    print("\nCleaning text...")
    df_playstore_clean['cleaned_text'] = df_playstore_clean['text'].apply(clean_text)
    
    # Filter data yang kosong setelah cleaning
    df_playstore_clean = df_playstore_clean[
        df_playstore_clean['cleaned_text'].str.len() > 0
    ].reset_index(drop=True)
    
    print(f"\n✓ Data Play Store setelah cleaning: {len(df_playstore_clean)} baris")
    
    if 'rating' in df_playstore_clean.columns:
        print(f"\nDistribusi rating:")
        print(df_playstore_clean['rating'].value_counts().sort_index())
    
    print(f"\nPreview data bersih:")
    display(df_playstore_clean.head())
    
except Exception as e:
    print(f"⚠️ Error loading Play Store data: {e}")
    df_playstore_clean = pd.DataFrame()

## 6. Load dan Clean Data YouTube

In [ ]:
print("=" * 80)
print("CLEANING DATA: Scraper Youtube CoreTax.csv")
print("=" * 80)

try:
    # Load data YouTube
    df_youtube = pd.read_csv(DATA_PATH + 'Scraper Youtube CoreTax.csv', encoding='utf-8')
    
    print(f"✓ Data YouTube loaded: {len(df_youtube)} baris")
    print(f"Kolom: {df_youtube.columns.tolist()}")
    print(f"\nPreview data:")
    display(df_youtube.head())
    
    # Rename kolom agar sesuai format
    df_youtube_clean = pd.DataFrame({
        'text': df_youtube['text'],
        'source': 'youtube'
    })
    
    # Tambahkan kolom date jika ada
    if 'date' in df_youtube.columns:
        df_youtube_clean['date'] = df_youtube['date']
    
    # Clean text
    print("\nCleaning text...")
    df_youtube_clean['cleaned_text'] = df_youtube_clean['text'].apply(clean_text)
    
    # Filter data yang kosong setelah cleaning
    df_youtube_clean = df_youtube_clean[
        df_youtube_clean['cleaned_text'].str.len() > 0
    ].reset_index(drop=True)
    
    print(f"\n✓ Data YouTube setelah cleaning: {len(df_youtube_clean)} baris")
    print(f"\nPreview data bersih:")
    display(df_youtube_clean.head())
    
except Exception as e:
    print(f"⚠️ Error loading YouTube data: {e}")
    df_youtube_clean = pd.DataFrame()

## 7. Kombinasi Semua Data

In [ ]:
print("=" * 80)
print("KOMBINASI SEMUA DATA")
print("=" * 80)

# List untuk menyimpan dataframe yang akan digabungkan
dfs_to_combine = []

# Tentukan kolom yang akan digunakan
# Kolom minimal: text, cleaned_text, source
base_columns = ['text', 'cleaned_text', 'source']

# Proses data existing
if len(df_existing) > 0:
    df_existing_processed = df_existing.copy()
    
    # Pastikan ada kolom cleaned_text
    if 'cleaned_text' not in df_existing_processed.columns:
        print("Membuat cleaned_text untuk data existing...")
        df_existing_processed['cleaned_text'] = df_existing_processed['text'].apply(clean_text)
    
    # Jika tidak ada kolom text, gunakan cleaned_text sebagai text
    if 'text' not in df_existing_processed.columns and 'cleaned_text' in df_existing_processed.columns:
        df_existing_processed['text'] = df_existing_processed['cleaned_text']
    
    # Pilih kolom yang ada
    cols_to_keep = [col for col in df_existing_processed.columns if col in base_columns or col in ['sentiment', 'sentiment_score', 'date']]
    df_existing_final = df_existing_processed[cols_to_keep].copy()
    dfs_to_combine.append(df_existing_final)
    print(f"✓ Data existing: {len(df_existing_final)} baris")

# Proses data Play Store
if len(df_playstore_clean) > 0:
    cols_to_keep = [col for col in df_playstore_clean.columns if col in base_columns or col in ['date', 'rating']]
    df_playstore_final = df_playstore_clean[cols_to_keep].copy()
    dfs_to_combine.append(df_playstore_final)
    print(f"✓ Data Play Store: {len(df_playstore_final)} baris")

# Proses data YouTube
if len(df_youtube_clean) > 0:
    cols_to_keep = [col for col in df_youtube_clean.columns if col in base_columns or col in ['date']]
    df_youtube_final = df_youtube_clean[cols_to_keep].copy()
    dfs_to_combine.append(df_youtube_final)
    print(f"✓ Data YouTube: {len(df_youtube_final)} baris")

# Gabungkan semua data
if len(dfs_to_combine) > 0:
    df_combined = pd.concat(dfs_to_combine, ignore_index=True)
    
    # Shuffle data
    df_combined = df_combined.sample(frac=1, random_state=42).reset_index(drop=True)
    
    print(f"\n✓ Data berhasil digabungkan!")
    print(f"Total data: {len(df_combined)} baris")
    
    print(f"\nDistribusi per source:")
    print(df_combined['source'].value_counts())
    
    if 'sentiment' in df_combined.columns:
        print(f"\nDistribusi sentiment:")
        print(df_combined['sentiment'].value_counts())
    
    print(f"\nPreview data gabungan:")
    display(df_combined.head(10))
    
else:
    print("⚠️ Tidak ada data untuk digabungkan")
    df_combined = pd.DataFrame()

## 8. Statistik Data

In [ ]:
print("=" * 80)
print("STATISTIK DATA")
print("=" * 80)

if len(df_combined) > 0:
    print(f"\nTotal data: {len(df_combined)} baris")
    print(f"\nKolom yang tersedia:")
    print(df_combined.columns.tolist())
    
    print(f"\nInfo data:")
    df_combined.info()
    
    print(f"\nStatistik panjang text:")
    df_combined['text_length'] = df_combined['text'].str.len()
    df_combined['cleaned_text_length'] = df_combined['cleaned_text'].str.len()
    
    print(df_combined[['text_length', 'cleaned_text_length']].describe())
    
    # Hapus kolom temporary
    df_combined = df_combined.drop(['text_length', 'cleaned_text_length'], axis=1)
else:
    print("⚠️ Tidak ada data untuk ditampilkan")

## 9. Export Data

In [ ]:
print("=" * 80)
print("EXPORT DATA")
print("=" * 80)

if len(df_combined) > 0:
    # Export combined file
    output_filename = DATA_PATH + 'CoreTax_All_Data_Combined_Clean.csv'
    df_combined.to_csv(output_filename, index=False)
    print(f"✓ Data gabungan exported to: {output_filename}")
    
    # Export individual files jika diperlukan
    if len(df_playstore_clean) > 0:
        df_playstore_clean.to_csv(DATA_PATH + 'cleaned_playstore_coretax.csv', index=False)
        print(f"✓ Play Store data exported to: {DATA_PATH}cleaned_playstore_coretax.csv")
    
    if len(df_youtube_clean) > 0:
        df_youtube_clean.to_csv(DATA_PATH + 'cleaned_youtube_coretax.csv', index=False)
        print(f"✓ YouTube data exported to: {DATA_PATH}cleaned_youtube_coretax.csv")
    
    print(f"\n✓ Semua file berhasil di-export!")
    
    # Download file (untuk Google Colab)
    try:
        from google.colab import files
        print(f"\nDownloading file...")
        files.download(output_filename)
        print(f"✓ File downloaded!")
    except:
        print(f"\n(File tersimpan di Google Drive: {output_filename})")
else:
    print("⚠️ Tidak ada data untuk di-export")

## 10. Summary

In [ ]:
print("=" * 80)
print("SUMMARY")
print("=" * 80)

print(f"\n✓ Data cleaning dan kombinasi selesai!")
print(f"\nFile output (tersimpan di Google Drive):")
print(f"  1. {DATA_PATH}CoreTax_All_Data_Combined_Clean.csv (MAIN FILE)")
print(f"  2. {DATA_PATH}cleaned_playstore_coretax.csv")
print(f"  3. {DATA_PATH}cleaned_youtube_coretax.csv")

if len(df_combined) > 0:
    print(f"\nTotal data bersih: {len(df_combined):,} baris")
    print(f"\nBreakdown per source:")
    for source, count in df_combined['source'].value_counts().items():
        percentage = (count / len(df_combined)) * 100
        print(f"  - {source}: {count:,} baris ({percentage:.2f}%)")